In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
prefix = "http://www.perverted-justice.com"
webpage_response = requests.get('http://www.perverted-justice.com/?archive=byUserVotes')

In [3]:
#stores the content of webpage we're scraping for links
webpage = webpage_response.content
soup = BeautifulSoup(webpage, "lxml") #lxml is faster than html.parser"

In [4]:
#stores all 'a' tags with attribute "id" equal to "pedoLink"
pedo_links = soup.findAll("a", {"id": "pedoLink"})
count = len(pedo_links)
print(count)

627


Bingo! That's the number of pedo links provided on website! Making progress.

In [7]:
links = []
# We take the href part of the a tag, then marry that to the prefix, which is the base part for each link
for a in pedo_links:
    links.append(prefix+a["href"])

In [11]:
#pedo_data:\n",
pedo_data = {}

# I want to pull all the span tags with attribute inText
# Then I want to pull the entire conversation 
for link in links:
    subpage = requests.get(link)
    pedo = BeautifulSoup(subpage.content, "lxml")
    meta_pedo_data= pedo.findAll("span", {"class": "inText"}) 
    print(meta_pedo_data)

#df = pd.DataFrame(pedo_data)
#print(turtle_df.head())"

[<span class="inText">Paul Short</span>, <span class="inText">34</span>, <span class="inText">fleet_captain_jaime_wolfe</span>, <span class="inText">Chicago, Illinois</span>, <span class="inText">sadlilgrrl</span>, <span class="inText">13 year old</span>, <span class="inText">girl</span>]
[<span class="inText">Michael J. Coffey</span>, <span class="inText">54</span>, <span class="inText">DavieWants2</span>, <span class="inText">Suffolk County, New York</span>, <span class="inText">SnapShotDeath</span>, <span class="inText">14 year old</span>, <span class="inText">boy</span>]
[<span class="inText">Joe</span>, <span class="inText">28</span>, <span class="inText">vamale_692005</span>, <span class="inText">Alexandria, Virginia</span>, <span class="inText">sweet_erin78</span>, <span class="inText">14 year old</span>, <span class="inText">girl</span>]


KeyboardInterrupt: 